# 6조 (17011709 정선아, 17011741 문성용, 17011742 김소영)

In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
from collections import defaultdict

import os

import numpy as np
import pandas as pd

from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = Dataset.load_builtin('ml-1m')
kf = KFold(n_splits = 5)
sim_options = {'name' : 'cosine', 'user_based' : True}

## 1) Precision & Recall & F1-measure

In [3]:
# https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py에서 가져온 코드
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [4]:
def get_P_R_F(data, algo):
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k = 5, threshold = 4)

        P = sum(prec for prec in precisions.values()) / len(precisions)
        R = sum(rec for rec in recalls.values()) / len(recalls)
        F1 = 2 * P * R / (P + R)

        print('precision : ', P)
        print('recall : ', R)
        print('F1 : '  , F1)

### CF with mean

In [5]:
algo = KNNWithMeans(k = 40, min_k = 1, simoptions = sim_options)
get_P_R_F(data, algo)

Computing the msd similarity matrix...
Done computing similarity matrix.
precision :  0.8962039644415173
recall :  0.2490751859424374
F1 :  0.3898126828045115
Computing the msd similarity matrix...
Done computing similarity matrix.
precision :  0.8911543772438719
recall :  0.2487028423850319
F1 :  0.38887787489132974
Computing the msd similarity matrix...
Done computing similarity matrix.
precision :  0.8958947600905677
recall :  0.2494088494133466
F1 :  0.3901918748102418
Computing the msd similarity matrix...
Done computing similarity matrix.
precision :  0.89377037405383
recall :  0.25096818574626323
F1 :  0.39189372513007664
Computing the msd similarity matrix...
Done computing similarity matrix.
precision :  0.8880987899884119
recall :  0.2517205559522374
F1 :  0.3922599172448734


### SVD

In [6]:
algo = SVD(n_factors = 100, n_epochs = 20, biased = False, lr_all = 0.005, reg_all = 0)
get_P_R_F(data, algo)

precision :  0.8611101910124881
recall :  0.2771596537950983
F1 :  0.4193469650613405
precision :  0.8609160178896915
recall :  0.27210717707825505
F1 :  0.413515678001709
precision :  0.8611267683466096
recall :  0.27760110523686166
F1 :  0.4198540286711317
precision :  0.8629411764706003
recall :  0.2765924951788015
F1 :  0.4189135593458251
precision :  0.8649314765694208
recall :  0.2790423324148288
F1 :  0.4219545844589454


### PMF

In [7]:
algo = SVD(n_factors = 100, n_epochs = 20, biased = False, lr_all = 0.005, reg_all = 0.02)
get_P_R_F(data, algo)

precision :  0.895736049014749
recall :  0.26456336312922696
F1 :  0.40847894797351236
precision :  0.8949304174950438
recall :  0.2653282415822242
F1 :  0.40930582530837656
precision :  0.8973243870112788
recall :  0.2651972224976996
F1 :  0.40939958994027487
precision :  0.9012170889220201
recall :  0.2623175644341266
F1 :  0.4063567356770022
precision :  0.9001574150787187
recall :  0.264476511509726
F1 :  0.40883317498230615


### PMF with bias

In [8]:
algo = SVD(n_factors = 100, n_epochs = 20, biased = True, lr_all = 0.005, reg_all = 0.02)
get_P_R_F(data, algo)

precision :  0.8807811291570121
recall :  0.28089336116659297
F1 :  0.42594646586773366
precision :  0.8849373309038829
recall :  0.2790187150746062
F1 :  0.4242670122182881
precision :  0.886203181968856
recall :  0.28260905579790874
F1 :  0.42855308390653163
precision :  0.886988231394013
recall :  0.28514419886197023
F1 :  0.43155456177525325
precision :  0.8838940448569345
recall :  0.2849378859430448
F1 :  0.43095143776027


## 2) NDCG

In [9]:
df = pd.DataFrame(data.raw_ratings, columns = ['uid', 'iid', 'rate', 'timestamp'])
del df['timestamp']
user = list(set(df.uid))

In [10]:
def addRating(uid, pred_data): 
    add = df[df.uid == uid] # 해당 사용자 행만 추출
    add['pred'] = 0.0 # 데이터프레임에 예측 레이팅 열 추가
    for iid, i in zip(add.iid, add.index):
        add.set_value(i, 'pred', pred_data.iloc[int(uid) - 1][int(iid) - 1]) # uid, iid 1부터 시작하므로 -1
    
    return add

def DCG(data):
    pred_sort = list(data.sort_values(by = ['pred'], axis = 0, ascending = False).rate)
    # 예측 레이팅을 기준으로 내림차순으로 정렬한 실제 레이팅 리스트
    dcg = pred_sort[0] # 예측 레이팅이 가장 높은 아이템의 실제 레이팅 저장
    for i in range(1, len(pred_sort)): # 리스트 돌며
        dcg += pred_sort[i] / np.log2(i + 1) # 순서대로 가중치를 줄여가며 더해주기
        
    return dcg

def IDCG(data):
    rate_sort = list(data.sort_values(by = ['rate'], axis = 0, ascending = False).rate)
    # 실제 레이팅을 기준으로 내림차순으로 정렬한 실제 레이팅 리스트
    idcg = rate_sort[0] # 실제 레이팅이 가장 높은 아이템의 실제 레이팅 저장
    for i in range(1, len(rate_sort)): # 리스트 돌며
        idcg += rate_sort[i] / np.log2(i + 1) # 순서대로 가중치를 줄여가며 더해주기
        
    return idcg

In [11]:
def NDCG(uid, pred_data): # 한 사용자의 NDCG
    table = addRating(uid, pred_data) # 예측 레이팅 열 추가한 데이터프레임
    dcg = DCG(table)
    idcg = IDCG(table)
    
    return dcg / idcg

In [12]:
def get_NDCG(user, pred_data): # 모든 사용자의(사용자 ID별) NDCG
    NDCG_ = {} # 딕셔너리 생성
    for u in user: # 각 유저를 돌며
        value = NDCG(u, pred_data) # NDCG 값 구하고
        NDCG_[u] = value # key로는 user_id, value로는 NDCG 값 저장
    return NDCG_;

In [13]:
# Assignment#4에서 구한 예측 레이팅 불러오기
pred_rating_cf_mean = pd.read_csv('pred_rating_cf_mean.csv')
pred_rating_svd = pd.read_csv('pred_rating_svd.csv')
pred_rating_pmf = pd.read_csv('pred_rating_pmf.csv')
pred_rating_pmf_bias = pd.read_csv('pred_rating_pmf_bias.csv')

### CF with mean

In [14]:
NDCG_CFwM = get_NDCG(user, pred_rating_cf_mean)

In [15]:
NDCG_CFwM

{'4588': 0.999906203245551,
 '5936': 0.9979333634608171,
 '904': 0.9868779484555891,
 '1630': 0.9958443992226842,
 '2431': 0.9994348586573718,
 '4718': 0.9976535451031814,
 '4808': 0.9893583247046808,
 '4534': 0.9989756866290592,
 '1892': 1.0,
 '5428': 0.996639860237544,
 '3129': 0.9896131941953706,
 '1740': 0.9981203274687444,
 '4743': 0.9961775543242999,
 '2770': 0.9997972405735034,
 '4496': 0.9940799538769675,
 '5052': 0.9978286159554615,
 '2009': 0.9996372580496933,
 '5940': 0.9994562427316394,
 '5737': 0.9988458763351157,
 '3418': 0.99617349349831,
 '4246': 0.9999222021918399,
 '2113': 0.9997636197647757,
 '3247': 1.0,
 '2606': 0.9993384585299036,
 '2493': 0.9979474287288014,
 '217': 1.0,
 '4179': 0.9981745690872658,
 '604': 0.9924369335871349,
 '1552': 0.9954221597809207,
 '2259': 0.9984631227922278,
 '3320': 0.9906826985071912,
 '3969': 0.9957297670939295,
 '2299': 0.9976721515020555,
 '5442': 0.99944767639187,
 '2707': 0.9998788708114181,
 '2723': 0.9977734619240484,
 '4856': 0

### SVD

In [16]:
NDCG_SVD = get_NDCG(user, pred_rating_svd)

In [17]:
NDCG_SVD

{'4588': 0.9967490033397999,
 '5936': 0.9996557191989026,
 '904': 0.9809220261143536,
 '1630': 0.9850135636609549,
 '2431': 0.9945841422978318,
 '4718': 0.9934984152115843,
 '4808': 0.9944729215401833,
 '4534': 0.9976453829031583,
 '1892': 0.9895549802859969,
 '5428': 0.996723924619148,
 '3129': 0.9880550986663326,
 '1740': 0.993563002892058,
 '4743': 0.9916632698247082,
 '2770': 0.999176789102823,
 '4496': 0.991227421298615,
 '5052': 0.984875519978155,
 '2009': 0.9846299016758809,
 '5940': 0.9823813772956574,
 '5737': 0.9950696686736665,
 '3418': 0.9927856534893139,
 '4246': 0.9919652070770867,
 '2113': 0.9865633323989448,
 '3247': 0.9935222630376032,
 '2606': 0.9944469355367652,
 '2493': 0.9942213517828615,
 '217': 0.9947898900316426,
 '4179': 0.9982433880259887,
 '604': 0.991219089385758,
 '1552': 0.9954494932709187,
 '2259': 0.996141623090061,
 '3320': 0.9924819078264869,
 '3969': 0.9880154676393047,
 '2299': 0.9933850194624483,
 '5442': 0.9855015991824713,
 '2707': 0.9886775257637

### PMF

In [18]:
NDCG_PMF = get_NDCG(user, pred_rating_pmf)

In [19]:
NDCG_PMF

{'4588': 0.9938835046444211,
 '5936': 0.9971215968563413,
 '904': 0.9775940024264704,
 '1630': 0.9624997454027234,
 '2431': 0.9973822419213738,
 '4718': 0.9927399471230094,
 '4808': 0.9906519914208342,
 '4534': 0.9954015093319915,
 '1892': 0.968315697073079,
 '5428': 0.9954061322053124,
 '3129': 0.9885887766298334,
 '1740': 0.9944378609256713,
 '4743': 0.9692758086733881,
 '2770': 0.9945479349698176,
 '4496': 0.9880792589160171,
 '5052': 0.9800402111893485,
 '2009': 0.9935199359223876,
 '5940': 0.9917983179393813,
 '5737': 0.9894827344878482,
 '3418': 0.9943727808842151,
 '4246': 0.9852878058867388,
 '2113': 0.9835470210875031,
 '3247': 0.9699043061780819,
 '2606': 0.9884087687738915,
 '2493': 0.9816311065811862,
 '217': 0.9651756368655827,
 '4179': 0.9965098428007823,
 '604': 0.9881419351635764,
 '1552': 0.9933050894910871,
 '2259': 0.9956655126217587,
 '3320': 0.9897993663753248,
 '3969': 0.9694471618906387,
 '2299': 0.9889864759189122,
 '5442': 0.9890067081031558,
 '2707': 0.9861977

### PMF with bias

In [23]:
NDCG_PMFwB = get_NDCG(user, pred_rating_pmf_bias)

In [24]:
NDCG_PMFwB

{'4588': 0.9906349597150415,
 '5936': 0.9923596607320979,
 '904': 0.9805516032326546,
 '1630': 0.9685579071506001,
 '2431': 0.9870053233511156,
 '4718': 0.9904962911324934,
 '4808': 0.9907252726193068,
 '4534': 0.9941046529467062,
 '1892': 0.9638046617758859,
 '5428': 0.995919418624657,
 '3129': 0.98832087156305,
 '1740': 0.9892755909127516,
 '4743': 0.9867672084880109,
 '2770': 0.9965422691734663,
 '4496': 0.9812055452628851,
 '5052': 0.9724572109583111,
 '2009': 0.9939212438668729,
 '5940': 0.9820472502051686,
 '5737': 0.9928863321089857,
 '3418': 0.9952525793718587,
 '4246': 0.9882418186230018,
 '2113': 0.9856281126212952,
 '3247': 0.9844591526193843,
 '2606': 0.9765066290070725,
 '2493': 0.9878405559682341,
 '217': 0.9829124067118632,
 '4179': 0.9973383289304772,
 '604': 0.9849887500846006,
 '1552': 0.9960272921464857,
 '2259': 0.9967230112624436,
 '3320': 0.9898881575177378,
 '3969': 0.966450558601186,
 '2299': 0.9917818354236585,
 '5442': 0.9892864636522647,
 '2707': 0.9797794201